# Figures

## Information
Project: **The price of being late: short- and long-term consequences of a delayed migration timing**  
Author: Iris Bontekoe  
Program: Python 3.8.5  
Description: This script contains all code to produce part of the figures in the manuscript.

## Preparation

In [ ]:
import os
import conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.basemap import Basemap
import datetime
#import matplotlib.animation as animation
from windrose import WindroseAxes
from matplotlib.colors import ListedColormap
import windrose
from matplotlib import gridspec

In [ ]:
# Define colors
ColorC = "#F8931D"
ColorA = "#2E3192"
ColorCC = "#C33184"

# Define names
NameC = "Naturally-timed"
NameCC = "Control"
NameA = "Delayed"

In [ ]:
# Set the path to the folder where the data is located and to the folder where the output should be saved
data_folder = "[...]"
output_folder = "[...]"

## Fig. 1A

In [ ]:
# Load the data
file_name_in = "Second_DataAff_All_Second.pkl"
dataA = pd.read_pickle(data_folder+file_name_in)

file_name_in = "Second_DataCASCB_All_Second.pkl"
dataC = pd.read_pickle(data_folder+file_name_in)

file_name_in = "Second_DataCC_All_Second.pkl"
dataCC = pd.read_pickle(data_folder+file_name_in)

In [ ]:
# Load the start and end of migration data
data = pd.read_csv(data_folder + "All_SegTimMinLat.csv",sep=",",low_memory=False)

# Convert the timestamps
data["MinLatDate1"] = pd.to_datetime(data["MinLatDate1"],format="%Y-%m-%d")

# Remove data without an MinLatDate1
data = data[~np.isnan(data["MinLatDate1"])]

# Add Individual and Aviary
data["Individual"] = data["Aviary"].map(str)+"_"+data["tag-local-identifier"].map(str)

In [ ]:
# Prepare the Affenberg data
dataA["Aviary"] = "Affenberg"
dataA["Individual"] = dataA["Aviary"].map(str)+"_"+dataA["tag-local-identifier"].map(str)

# Add EndMigFinal to the location data
dataA["MinLatDate1"] = dataA["Individual"].map(data.set_index("Individual")["MinLatDate1"].to_dict())
dataA["Distance1"] = dataA["Individual"].map(data.set_index("Individual")["Distance1"].to_dict())

# Remove data after EndMigFinal
dataA = dataA[dataA["timestamp"]<dataA["MinLatDate1"]]

# Remove individuals that died before finishing migration
died = [7023,7027,7028,7962,7964,7967,7990,8039]
dataA = dataA[~dataA["tag-local-identifier"].isin(died)]

In [ ]:
# Prepare the CareCenter data
dataCC["Aviary"] = "CareCenter"
dataCC["Individual"] = dataCC["Aviary"].map(str)+"_"+dataCC["tag-local-identifier"].map(str)

# Add EndMigFinal to the location data
dataCC["MinLatDate1"] = dataCC["Individual"].map(data.set_index("Individual")["MinLatDate1"].to_dict())
dataCC["Distance1"] = dataCC["Individual"].map(data.set_index("Individual")["Distance1"].to_dict())

# Remove individuals without an end of migration
dataCC = dataCC[~np.isnan(dataCC["MinLatDate1"])]

# Remove data after EndMigFinal
dataCC = dataCC[dataCC["timestamp"]<dataCC["MinLatDate1"]]

In [ ]:
# Prepare the CASCB data
dataC["Aviary"] = "CASCB"
dataC["Individual"] = dataC["Aviary"].map(str)+"_"+dataC["tag-local-identifier"].map(str)

# Add EndMigFinal to the location data
dataC["MinLatDate1"] = dataC["Individual"].map(data.set_index("Individual")["MinLatDate1"].to_dict())
dataC["Distance1"] = dataC["Individual"].map(data.set_index("Individual")["Distance1"].to_dict())

# Remove individuals without an end of migration
dataC = dataC[~np.isnan(dataC["MinLatDate1"])]

# Remove data after EndMigFinal
dataC = dataC[dataC["timestamp"]<dataC["MinLatDate1"]]

In [ ]:
# Find average coordinates
MaxLat=max([dataA["location-lat"].max(),dataC["location-lat"].max(),dataCC["location-lat"].max()])
MaxLong=max([dataA["location-long"].max(),dataC["location-long"].max(),dataCC["location-long"].max()])
MinLat=min([dataA["location-lat"].min(),dataC["location-lat"].min(),dataCC["location-lat"].min()])
MinLong=min([dataA["location-long"].min(),dataC["location-long"].min(),dataCC["location-long"].min()])

In [ ]:
# Get the number of individuals for each study
print("Affenberg:",len(dataA["Individual"].unique()))
print("CareCenter:",len(dataCC["Individual"].unique()))
print("CASCB:",len(dataC["Individual"].unique()))

In [ ]:
fig = plt.figure()
fig = plt.figure(figsize=(30,22))
plt.rc("font",size=35)

spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1,1,1], wspace=0.02,
                         hspace=0.005, height_ratios=[12,6])

#-------------#
#- CASCB map -#
#-------------#

ax0 = fig.add_subplot(spec[0])
#ax0.set_title(NameC,pad=15)
map = Basemap(projection="lcc", resolution="h",
            lat_0=sum([MinLat,MaxLat])/2, lon_0=sum([MinLong,MaxLong])/2,
            width=3.5E6, height=4.8E6)

map.shadedrelief()
map.drawcountries(color="white")
map.drawparallels(np.arange(0,70,10),labels=[1,0,0,0],color="#7F7F7F",textcolor="#7F7F7F")

map.scatter(sum([MinLong,MaxLong])/2,sum([MinLat,MaxLat])/2,latlon=True,s=5000000,c="white",alpha=0.5)

for i in dataC["tag-local-identifier"].unique():
    map.plot(dataC[dataC["tag-local-identifier"]==i]["location-long"], dataC[dataC["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorC, 
            linewidth=3,
            alpha=0.75,
            zorder=10
            )

for i in dataC["tag-local-identifier"].unique():
    map.scatter(dataC[dataC["tag-local-identifier"]==i].iloc[-1]["location-long"],dataC[dataC["tag-local-identifier"]==i].iloc[-1]["location-lat"],
            latlon=True,
            c="white",
            s=75,
            zorder=15
            )

        
#------------------#
#- CareCenter map -#
#------------------#

ax1 = fig.add_subplot(spec[1])
#ax1.set_title(NameCC,pad=15)
map = Basemap(projection="lcc", resolution="h",
            lat_0=sum([MinLat,MaxLat])/2, lon_0=sum([MinLong,MaxLong])/2,
            width=3.5E6, height=4.8E6)

map.shadedrelief()
map.drawcountries(color="white")
map.drawparallels(np.arange(0,70,10),labels=[0,0,0,0],color="#7F7F7F",textcolor="#7F7F7F")

map.scatter(sum([MinLong,MaxLong])/2,sum([MinLat,MaxLat])/2,latlon=True,s=5000000,c="white",alpha=0.5)

for i in dataCC["tag-local-identifier"].unique():
    map.plot(dataCC[dataCC["tag-local-identifier"]==i]["location-long"], dataCC[dataCC["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorCC, 
            linewidth=3,
            alpha=0.75,
            zorder=10
            )
        
for i in dataCC["tag-local-identifier"].unique():
    map.scatter(dataCC[dataCC["tag-local-identifier"]==i].iloc[-1]["location-long"],dataCC[dataCC["tag-local-identifier"]==i].iloc[-1]["location-lat"],
            latlon=True,
            c="white",
            s=75,
            zorder=15
            )
            

#-----------------#
#- Affenberg map -#
#-----------------#

ax2 = fig.add_subplot(spec[2])
#ax2.set_title(NameA,pad=15)
map = Basemap(projection="lcc", resolution="h",
            lat_0=sum([MinLat,MaxLat])/2, lon_0=sum([MinLong,MaxLong])/2,
            width=3.5E6, height=4.8E6)

map.shadedrelief()
map.drawcountries(color="white")
map.drawparallels(np.arange(0,70,10),labels=[0,1,0,0],color="#7F7F7F",textcolor="#7F7F7F")

map.scatter(sum([MinLong,MaxLong])/2,sum([MinLat,MaxLat])/2,latlon=True,s=5000000,c="white",alpha=0.5)

for i in dataA["tag-local-identifier"].unique():
    map.plot(dataA[dataA["tag-local-identifier"]==i]["location-long"], dataA[dataA["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorA, 
            linewidth=3,
            alpha=0.75,
            zorder=10
            )
        
for i in dataA["tag-local-identifier"].unique():            
    map.scatter(dataA[dataA["tag-local-identifier"]==i].iloc[-1]["location-long"],dataA[dataA["tag-local-identifier"]==i].iloc[-1]["location-lat"],
            latlon=True,
            c="white",
            s=75,
            zorder=15
            )

# Save the maps
plt.savefig(output_folder+"F1A.svg",bbox_inches='tight')

## Fig. 1A2

In [ ]:
# Load the data
file_name_in = "DataAff_TempWind_S.pkl"
dataA2 = pd.read_pickle(data_folder+file_name_in)

file_name_in = "DataCC_TempWind_S.pkl"
dataCC2 = pd.read_pickle(data_folder+file_name_in)

file_name_in = "DataCASCB_TempWind_S.pkl"
dataC2 = pd.read_pickle(data_folder+file_name_in)

In [ ]:
fig = plt.figure(figsize=(17.5,15))
plt.rc("font",size=65)

m = Basemap(projection="lcc", resolution="h",
            lat_0=45.75, lon_0=4.8,
            width=8.25E5, height=8E5)

m.shadedrelief()
m.drawcountries(color='white')
m.drawparallels(np.arange(47.5,70,20),labels=[1,1,0,0],linewidth=2.5,color="#7F7F7F",textcolor="#7F7F7F")
m.drawparallels(np.arange(44.0,70,20),labels=[1,1,0,0],linewidth=2.5,color="#7F7F7F",textcolor="#7F7F7F")

m.scatter(4.8,45.75,latlon=True,s=8000000,c="white",alpha=0.5)

for i in dataC2["tag-local-identifier"].unique():
    m.plot(dataC2[dataC2["tag-local-identifier"]==i]["location-long"],dataC2[dataC2["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorC,
            linewidth=3,
            alpha=0.5
          )
        
for i in dataCC2["tag-local-identifier"].unique():
    m.plot(dataCC2[dataCC2["tag-local-identifier"]==i]["location-long"],dataCC2[dataCC2["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorCC,
            linewidth=3,
            alpha=0.5
          )            

for i in dataA2["tag-local-identifier"].unique():
    m.plot(dataA2[dataA2["tag-local-identifier"]==i]["location-long"],dataA2[dataA2["tag-local-identifier"]==i]["location-lat"],
            latlon=True,
            c=ColorA,
            linewidth=3,
            alpha=0.5
          )

plt.savefig(output_folder+"F1A2.svg",bbox_inches='tight')

## Fig. 3B

In [ ]:
file_name_in = "DataAff_TempWind_R.pkl"
data = pd.read_pickle(data_folder+file_name_in)

In [ ]:
# Subset the data to only contain one burst
data = data[data["tag-local-identifier"]==7004]
data = data[data["timestamp"]>datetime.datetime(2019,9,16,9,0,7)]
data = data[data["timestamp"]<datetime.datetime(2019,9,16,9,10,8)]

In [ ]:
fig = plt.figure(figsize=(16.8,13.3))

plt.scatter(data["location-long"],data["location-lat"],c=data["height-above-ellipsoid"],s=80)
plt.axis('off')
plt.axis("tight")
plt.gca().invert_xaxis()

plt.savefig(output_folder+"F3B.svg",bbox_inches='tight',transparent=True)

## Fig. 3D

In [ ]:
# Write function to plot wind roses
def WindRosePlot():

    # Load data
    data = pd.read_pickle(data_folder+file_name_in)
    
    # Only keep the wanted flight type
    if FlightType == "gliding":
        data=data[~data["GlidingID"].isna()]
    elif FlightType == "climbing":
        data=data[~data["ClimbingID"].isna()]
    
    # Plot the figure
    plt.rc('font',size=35)
    fig = plt.figure(figsize=(6.65693,6.65693), dpi=80, facecolor='w', edgecolor='w')
    rect = [0.1, 0.1, 0.75, 0.75]
    ax = WindroseAxes(fig, rect)
    fig.add_axes(ax)

    # Get the colormap colors
    cmap = ["#7fcdbb", "#1d91c0", "#253494", "#161F58"]
    
    cmap = ListedColormap(cmap)

    ax.bar(data[Angle_bar], data[Speed_bar], normed=True, opening=1, edgecolor='black',linewidth=0.1,bins=np.arange(0, 17, 5),cmap=cmap,zorder=5,alpha=0.6)
    ax.set_yticks(np.arange(5, 35, step=10))
    ax.xaxis.set_tick_params(pad=25)
    
    # Group data by day
    data = data.groupby(["Day","tag-local-identifier"],as_index=False).agg(
        WindSpeed_PL = ("WindSpeed_PL",'mean'),
        WindDirection_PL = ("WindDirection_PL",'mean'),
        heading = ("heading",'mean'),
        ground_speed = ("ground-speed",'mean')
    )
    data.rename(columns={"ground_speed":"ground-speed"},inplace=True)
    
    # Add dots to the plot
    Headings = np.radians(np.array(data[Angle_dots]))
    windrose.wrscatter(Headings,len(Headings)*[30],ax=ax,alpha=0.25,s=100,color=Color,zorder=10)
    
    # Add a label to the plot
    ax.text(np.radians(90),20.5,Label,color="white",va="center",ha="center",size=50,zorder=15)
    ax.text(np.radians(90),21,"D",color=Color,va="center",ha="center",size=50,bbox=dict(boxstyle="round",facecolor=Color,edgecolor=Color,pad=0.45))
    
    if LegendPresent == True:
        #ax.legend(prop={'size': 30},bbox_to_anchor=(-0.2,0.725),loc="right",title="Wind speed (m/s)")
        ax.legend(prop={'size': 30},bbox_to_anchor=(1.5,1.175),loc="lower center",title="Wind speed (m/s)",ncol=5,labels=[1,2,3,4])
        
    ax.grid(color="Grey",alpha=0.1,zorder=1)
    
    plt.savefig(output_folder+file_name_out+".svg",bbox_inches='tight',transparent=True)
    plt.show()

In [ ]:
# Define objects
file_name_in = "DataAff_TempWind_S.pkl"
Aviary = "Affenberg"
Angle_bar = "WindDirection_PL"
Speed_bar = "WindSpeed_PL"
Angle_dots = "heading"
Speed_dots = "ground-speed"
FlightType = "gliding"
Color = ColorA
file_name_out = "F3D3"
PlotTitle = "Delayed"
LegendPresent = False
Label = "D"

# Execute the function and print the run time
start = datetime.datetime.now()
WindRosePlot()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

In [ ]:
# Define objects
file_name_in = "DataCC_TempWind_S.pkl"
Aviary = "CareCenter"
Angle_bar = "WindDirection_PL"
Speed_bar = "WindSpeed_PL"
Angle_dots = "heading"
Speed_dots = "ground-speed"
FlightType = "gliding"
Color = ColorCC
file_name_out = "F3D2"
PlotTitle = "Control"
LegendPresent = False
Label = "C"

# Execute the function and print the run time
start = datetime.datetime.now()
WindRosePlot()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

In [ ]:
# Define objects
file_name_in = "DataCASCB_TempWind_S.pkl"
Aviary = "CASCB"
Angle_bar = "WindDirection_PL"
Speed_bar = "WindSpeed_PL"
Angle_dots = "heading"
Speed_dots = "ground-speed"
FlightType = "gliding"
Color = ColorC
file_name_out = "F3D1"
PlotTitle = "Naturally-timed"
LegendPresent = True
Label = "N"

# Execute the function and print the run time
start = datetime.datetime.now()
blabla = WindRosePlot()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)